In [1]:
import gymnasium as gym
import ray
from ray import tune
from ray.tune.registry import register_env

from simulation import SimulationEnv

register_env('trading_env-v0', lambda config: SimulationEnv(**config))

In [2]:
ray.init()

2023-05-02 22:58:24,632	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.11
Ray version:,2.4.0
Dashboard:,http://127.0.0.1:8265


In [18]:
from ray.rllib.algorithms.dqn import DQNConfig
from ray.rllib.algorithms.ppo import PPOConfig
from ray.tune.logger import pretty_print

dqn = (
    PPOConfig()
    .rollouts(num_rollout_workers=1)
    .resources(num_gpus=1)
    .environment(env='CartPole-v1')
    .build()
)

for i in range(50):
    result = dqn.train()
    print(pretty_print(result))

2023-05-03 00:06:10,163	WARNING util.py:67 -- Install gputil for GPU system monitoring.
(RolloutWorker pid=371580) /home/fassty/anaconda3/envs/diploma_thesis/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
(RolloutWorker pid=371580)   if not isinstance(terminated, (bool, np.bool8)):


agent_timesteps_total: 4000
connector_metrics:
  ObsPreprocessorConnector_ms: 0.003806880263031506
  StateBufferConnector_ms: 0.0031102550485746457
  ViewRequirementAgentConnector_ms: 0.07859175322485752
counters:
  num_agent_steps_sampled: 4000
  num_agent_steps_trained: 4000
  num_env_steps_sampled: 4000
  num_env_steps_trained: 4000
custom_metrics: {}
date: 2023-05-03_00-06-19
done: false
episode_len_mean: 21.80327868852459
episode_media: {}
episode_reward_max: 67.0
episode_reward_mean: 21.80327868852459
episode_reward_min: 8.0
episodes_this_iter: 183
episodes_total: 183
hostname: seymour
info:
  learner:
    default_policy:
      custom_metrics: {}
      diff_num_grad_updates_vs_sampler_policy: 464.5
      learner_stats:
        cur_kl_coeff: 0.20000000000000004
        cur_lr: 5.0000000000000016e-05
        entropy: 0.6642202406160294
        entropy_coeff: 0.0
        kl: 0.02892561394374421
        policy_loss: -0.04225908713715692
        total_loss: 8.885406422358686
        v

In [10]:
env = gym.make('CartPole-v1', render_mode='human')

In [16]:
state, info = env.reset()

done = False
return_ = 0
while not done:
    action = dqn.compute_single_action(state)
    state, reward, terminated, truncated, _ = env.step(action)
    done = terminated or truncated

    return_ += reward
    env.render()
print(return_)

92.0


In [12]:
!pip install "gymnasium[classic_control]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 22.6 MB/s eta 0:00:0000:0100:01
